Course Human-Centered Data Science ([HCDS](https://www.mi.fu-berlin.de/en/inf/groups/hcc/teaching/winter_term_2020_21/course_human_centered_data_science.html)) - Winter Term 2020/21 - [HCC](https://www.mi.fu-berlin.de/en/inf/groups/hcc/index.html) | [Freie Universität Berlin](https://www.fu-berlin.de/)

***

# A2 - Wikipedia, ORES, and Bias in Data
Please follow the reproducability workflow as practiced during the last exercise.

## Step 1⃣ | Data acquisition

You will use two data sources: (1) Wikipedia articles of politicians and (2) world population data.

**Wikipedia articles -**
The Wikipedia articles can be found on [Figshare](https://figshare.com/articles/Untitled_Item/5513449). It contains politiciaans by country from the English-language wikipedia. Please read through the documentation for this repository, then download and unzip it to extract the data file, which is called `page_data.csv`.

**Population data -**
The population data is available in `CSV` format in the `_data` folder. The file is named `export_2019.csv`. This dataset is drawn from the [world population datasheet](https://www.prb.org/international/indicator/population/table/) published by the Population Reference Bureau (downloaded 2020-11-13 10:14 AM). I have edited the dataset to make it easier to use in this assignment. The population per country is given in millions!

As the first step we need to import the neccessary libaries for this project.

In [1]:
import pandas as pd
import os

We now load the csv files as pandas dataframes for further processing.

In [2]:
population_df = pd.read_csv('../data_raw/export_2019.csv', delimiter=';')
articles_df = pd.read_csv('../data_raw/page_data.csv')

In [3]:
# load all files from a folder
# files = [file for file in os.listdir('../data_raw')]
# combined = pd.DataFrame()

# for file in files:
#     delimiter = ',' if file != 'export_2019.csv' else ';'
#     current_df = pd.read_csv('../data_raw/' + file, delimiter=delimiter)
#     combined = pd.concat([combined, current_df])
# combined

## Step 2⃣ | Data processing and cleaning
The data in `page_data.csv` contain some rows that you will need to filter out. It contains some page names that start with the string `"Template:"`. These pages are not Wikipedia articles, and should not be included in your analysis. The data in `export_2019.csv` does not need any cleaning.

***

| | `page_data.csv` | | |
|-|------|---------|--------|
| | **page** | **country** | **rev_id** |
|0|	Template:ZambiaProvincialMinisters | Zambia | 235107991 |
|1|	Bir I of Kanem | Chad | 355319463 |

***

| | `export_2019.csv` | | |
|-|------|---------|--------|
| | **country** | **population** | **region** |
|0|	Algeria | 44.357 | AFRICA |
|1|	Egypt | 100.803 | 355319463 |

***

We clean the Wikipedia articles of every row containing the `Template:` prefix.

In [4]:
articles_df = articles_df[~articles_df['page'].str.contains('Template:')]
articles_df

# sort out nan's
# new_df = data_df[data_df.isna().any(axis=1)]
# df.dropna(how='all')

# convert to int
# df['col'] = pd.to_numeric(df['col'])

# run function on df
# def get_city(address):
#   return address.split(',')[1]
# df['col'].apply(lambda addr: get_city(addr))

# python f strings
# .apply(lambda addr: f"{get_city(addr)} xy")

# plot labeling
# plt.xticks(df['x'].unique(), rotation='vertical', size=8)

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568
...,...,...,...
47192,Yahya Jammeh,Gambia,807482007
47193,Lucius Fairchild,United States,807483006
47194,Fahd of Saudi Arabia,Saudi Arabia,807483153
47195,Francis Fessenden,United States,807483270


### Getting article quality predictions with ORES

Now you need to get the predicted quality scores for each article in the Wikipedia dataset. We're using a machine learning system called [**ORES**](https://www.mediawiki.org/wiki/ORES) ("Objective Revision Evaluation Service"). ORES estimates the quality of an article (at a particular point in time), and assigns a series of probabilities that the article is in one of the six quality categories. The options are, from best to worst:

| ID | Quality Category |  Explanation |
|----|------------------|----------|
| 1 | FA    | Featured article |
| 2 | GA    | Good article |
| 3 | B     | B-class article |
| 4 | C     | C-class article |
| 5 | Start | Start-class article |
| 6 | Stub  | Stub-class article |

For context, these quality classes are a sub-set of quality assessment categories developed by Wikipedia editors. If you're curious, you can [read more](https://en.wikipedia.org/wiki/Wikipedia:Content_assessment#Grades) about what these assessment classes mean on English Wikipedia. For this assignment, you only need to know that these categories exist, and that ORES will assign one of these six categories to any `rev_id`. You need to extract all `rev_id`s in the `page_data.csv` file and use the ORES API to get the predicted quality score for that specific article revision.

### ORES REST API endpoint

The [ORES REST API](https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context_revid_model) is configured fairly similarly to the pageviews API we used for the last assignment. It expects the following parameters:

* **project** --> `enwiki`
* **revid** --> e.g. `235107991` or multiple ids e.g.: `235107991|355319463` (batch)
* **model** --> `wp10` - The name of a model to use when scoring.

**❗Note on batch processing:** Please read the documentation about [API usage](https://www.mediawiki.org/wiki/ORES#API_usage) if you want to query a large number of revisions (batches). 

You will notice that ORES returns a prediction value that contains the name of one category (e.g. `Start`), as well as probability values for each of the six quality categories. For this assignment, you only need to capture and use the value for prediction.

**❗Note:** It's possible that you will be unable to get a score for a particular article. If that happens, make sure to maintain a log of articles for which you were not able to retrieve an ORES score. This log should be saved as a separate file named `ORES_no_scores.csv` and should include the `page`, `country`, and `rev_id` (just as in `page_data.csv`).

You can use the following **samle code for API calls**:

In [17]:
import requests
import json
import numpy as np
from ratelimit import limits

# Customize these with your own information
headers = {
    'User-Agent': 'https://github.com/Arne117',
    'From': 'arner92@zedat.fu-berlin.de'
}

# 50 revisions within a given request, up to 4 parallel requests.
@limits(calls=4, period=0.1)
def get_ores_data(rev_ids, headers):
    
    # Define the endpoint
    # https://ores.wikimedia.org/scores/enwiki/?models=wp10&revids=807420979|807422778
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'

    params = {
        'project' : 'enwiki',
        'model'   : 'wp10',
        'revids'  : rev_ids
    }

    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
    data = json.loads(json.dumps(response))

    return data

result = get_ores_data('355319463|393276188|393822005|395521877|395526568|401577829|442937236|448555418|470173494|477962574|492060822|492964343|498683267|502721672|516633096|521986779|532253442|543225630|545936100|546364151|549300521|550682925|550953646|559553872|559788982|560758943|561744402|564873005|565745353|565745365|565745375|566504165|573710096|574571582|576988466|585894477|592289232|595693452|596181202|598819900|601122766|601127343|614786300|623004627|623334577|624468970|625509885|626606789|627001041|627051151', headers)
del result['enwiki']['models']

test_df = pd.DataFrame(result['enwiki'])
test_df.columns = ['score']
test_df.index.name = 'rev_id'
test_df['score'] = test_df['score'].apply(lambda score: score['wp10'])
test_df

# The last rows index gives the length of the df and is dived by 50 to get the number of chunks needed to fit 50 rev_ids into one chunk for the api call
#for chunk in np.array_split(articles_df['rev_id'], int(articles_df.iloc[-1].name / 50)):
#    print('|'.join(map(str, chunk)))

,scores
355319463,"{'wp10': {'score': {'prediction': 'Stub', 'pro..."
393276188,"{'wp10': {'score': {'prediction': 'Stub', 'pro..."
393822005,"{'wp10': {'score': {'prediction': 'Stub', 'pro..."
395521877,"{'wp10': {'score': {'prediction': 'Stub', 'pro..."
395526568,"{'wp10': {'score': {'prediction': 'Stub', 'pro..."
401577829,"{'wp10': {'score': {'prediction': 'Stub', 'pro..."
442937236,"{'wp10': {'score': {'prediction': 'Stub', 'pro..."
448555418,"{'wp10': {'score': {'prediction': 'Stub', 'pro..."
470173494,"{'wp10': {'score': {'prediction': 'Start', 'pr..."
477962574,"{'wp10': {'score': {'prediction': 'Stub', 'pro..."


In [43]:
test_df = pd.DataFrame(result['enwiki'])
test_df.columns = ['score']
test_df.index.name = 'rev_id'
test_df['score'] = test_df['score'].apply(lambda score: score['wp10'])
test_df

,score
rev_id,
355319463,"{'score': {'prediction': 'Stub', 'probability'..."
393276188,"{'score': {'prediction': 'Stub', 'probability'..."
393822005,"{'score': {'prediction': 'Stub', 'probability'..."
395521877,"{'score': {'prediction': 'Stub', 'probability'..."
395526568,"{'score': {'prediction': 'Stub', 'probability'..."
401577829,"{'score': {'prediction': 'Stub', 'probability'..."
442937236,"{'score': {'prediction': 'Stub', 'probability'..."
448555418,"{'score': {'prediction': 'Stub', 'probability'..."
470173494,"{'score': {'prediction': 'Start', 'probability..."


Sending one request for each `rev_id` might take some time. If you want to send batches you can use `'|'.join(str(x) for x in revision_ids` to put your ids together. Please make sure to deal with [exception handling](https://www.w3schools.com/python/python_try_except.asp) of the `KeyError` exception, when extracting the `prediction` from the `JSON` response.

### Combining the datasets

Now you need to combine both dataset: (1) the wikipedia articles and its ORES quality scores and (2) the population data. Both have columns named `country`. After merging the data, you'll invariably run into entries which cannot be merged. Either the population dataset does not have an entry for the equivalent Wikipedia country, or vis versa.

Please remove any rows that do not have matching data, and output them to a `CSV` file called `countries-no_match.csv`. Consolidate the remaining data into a single `CSV` file called `politicians_by_country.csv`.

The schema for that file should look like the following table:


| article_name | country | region | revision_id | article_quality | population |
|--------------|---------|--------|-------------|-----------------|------------|
| Bir I of Kanem | Chad  | AFRICA | 807422778 | Stub | 16877000 |

## Step 3⃣ | Analysis

Your analysis will consist of calculating the proportion (as a percentage) of articles-per-population (we can also call it `coverage`) and high-quality articles (we can also call it `relative-quality`)for **each country** and for **each region**. By `"high quality"` arcticle we mean an article that ORES predicted as `FA` (featured article) or `GA` (good article).

**Examples:**

* if a country has a population of `10,000` people, and you found `10` articles about politicians from that country, then the percentage of `articles-per-population` would be `0.1%`.
* if a country has `10` articles about politicians, and `2` of them are `FA` or `GA` class articles, then the percentage of `high-quality-articles` would be `20%`.

### Results format

The results from this analysis are six `data tables`. Embed these tables in the Jupyter notebook. You do not need to graph or otherwise visualize the data for this assignment. The tables will show:

1. **Top 10 countries by coverage**<br>10 highest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Bottom 10 countries by coverage**<br>10 lowest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Top 10 countries by relative quality**<br>10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Bottom 10 countries by relative quality**<br>10 lowest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population
1. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

**❗Hint:** You will find what country belongs to which region (e.g. `ASIA`) also in `export_2019.csv`. You need to calculate the total poulation per region. For that you could use `groupby` and also check out `apply`.

***

#### Credits

This exercise is slighty adapted from the course [Human Centered Data Science (Fall 2019)](https://wiki.communitydata.science/Human_Centered_Data_Science_(Fall_2019)) of [Univeristy of Washington](https://www.washington.edu/datasciencemasters/) by [Jonathan T. Morgan](https://wiki.communitydata.science/User:Jtmorgan).

Same as the original inventors, we release the notebooks under the [Creative Commons Attribution license (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/).